In [1]:
import numpy as np, matplotlib.pyplot as plt, matplotlib as mpl
import pandas as pd
from astropy.io import fits
from astropy.table import Table
from drizzlepac import sky
import seaborn as sns
from tqdm import tqdm
%matplotlib inline

The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    
The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       imagefindpars           mapreg              photeq       
     pixreplace           pixtopix            pixtosky        refimagefindpars  
     resetbits          runastrodriz            sky               skytopix      
     tweakback            tweakreg           updatenpol


In [2]:
import os, sys

In [13]:
def compute_sky(input_image, n=100, good_frac=0.25, verbose=True):
    image = fits.getdata(input_image, 'SCI')
    flags = fits.getdata(input_image, 'FLAGS')
    meds = []
    for i in tqdm(range(0, image.shape[0]+n, n), disable=(not verbose)):
        for j in range(0, image.shape[1]+n, n):
            s1 = slice(i, i+n)
            s2 = slice(j, j+n)
            sub_im = image[s1, s2]
            n_total = sub_im.shape[0] * sub_im.shape[1]
            sub_mask = (flags[s1, s2] == 0)
            data = sub_im[sub_mask]
            data = data[~np.isnan(data)]
            n_final = len(data)
            if (n_final >= good_frac*n_total)&(n_final > 0):
                meds.append(np.median(data))
    return np.min(meds)

In [14]:
files = ['../data/NGC3377/reduced/NGC3377_F475W_drc.fits', '../data/NGC3377/reduced/NGC3377_F850LP_drc.fits',
        '../data/NGC4993/reduced/NGC4993_F475W_drc.fits', '../data/NGC4993/reduced/NGC4993_F850LP_drc.fits',
        '../data/M49/reduced/M49_F475W_drc.fits', '../data/M49/reduced/M49_F850LP_drc.fits',
        '../data/M87/reduced/M87_F475W_drc.fits', '../data/M87/reduced/M87_F606W_drc.fits', '../data/M87/reduced/M87_F814W_drc.fits']

In [15]:
for f in files:
    sky = compute_sky(f, n=100, verbose=False)
    print(f.rpartition('/')[-1], sky)

NGC3377_F475W_drc.fits 81.75273
NGC3377_F850LP_drc.fits 122.83262
NGC4993_F475W_drc.fits 170.75229
NGC4993_F850LP_drc.fits 58.188858
M49_F475W_drc.fits 68.45227
M49_F850LP_drc.fits 92.19022
M87_F475W_drc.fits 81.13346
M87_F606W_drc.fits 828.9583
M87_F814W_drc.fits 629.1317
